In [18]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
import operator
from qiskit.visualization import plot_histogram
from fractions import Fraction
from math import gcd

In [19]:
from qiskit import QuantumCircuit


def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2, 4, 7, 8, 11, 13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for _ in range(power):
        if a in [2, 13]:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        if a in [7, 8]:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        if a in [4, 11]:
            U.swap(1, 3)
            U.swap(0, 2)
        if a in [7, 11, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

In [20]:
def order_finding(N, a):

    n = len(bin(N)[2:])
    ancilla = 2 * n
    qr = QuantumRegister(n + ancilla)
    cr = ClassicalRegister(ancilla)
    qc = QuantumCircuit(qr, cr)

    qc.h(qr[:ancilla])
    qc.x(qr[ancilla])
    for q in range(ancilla):
        qc.compose(
            c_amod15(a, 2**q), [q] + [i + ancilla for i in range(n)], inplace=True
        )
    qc.compose(QFT(ancilla, inverse=True), range(ancilla), inplace=True)

    qc.measure(qr[:ancilla], cr)

    # display(qc.draw("mpl", fold=-1))

    simulator = AerSimulator()
    isa_circuit = transpile(qc, simulator)
    result = simulator.run(isa_circuit).result()
    counts = result.get_counts()

    # display(plot_histogram(counts))

    highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0]
    phase = int(highest_probability_outcome, 2) / 2**ancilla  # theta = s / r
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    return phase, r


a = 7
N = 15

FACTOR_FOUND = False
ATTEMPT = 0
while True:
    ATTEMPT += 1
    print(f"ATTEMPT {ATTEMPT}:")
    phase, r = order_finding(N, a)
    if phase != 0:
        x = (a ** (r // 2)) % N
        guesses = [gcd(x - 1, N), gcd(x + 1, N)]
        factors = []
        for guess in guesses:
            if guess not in [1, N] and (N % guess) == 0:
                factors.append(guess)
                FACTOR_FOUND = True
        if FACTOR_FOUND:
            print(f"Factors: {factors}")
            break

ATTEMPT 1:
Factors: [3]
